<a href="https://colab.research.google.com/github/ataraxia90/ataraxia90.github.io/blob/master/SRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # 느낌표를 붙이면 Shell 명령어로 실행됩니다.
# !python --version

# # Selenium을 확인하려면 다음과 같이 사용해야 합니다.
# import selenium
# print(selenium.__version__)

In [ ]:
!pip install selenium
!pip install telepot
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for telepot: filename=telepot-12.7-py3-none-any.whl size=57939 sha256=e0170db784b5d7f4ebaedff95b715ceba998fc69193ad1752fe21b7723e09511
  Stored in directory: /root/.cache/pip/wheels/9f/9a/92/2e34a093e40a09338847e54dc9fcaab4ce01a59ba17c784c6c
Successfully built telepot
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-upd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [ ]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait

import time
from random import randint
import telepot

# 텔레그램 메시지 발송
def tele(msg):
    token='6410808052:AAET4SYEsLM6iDZnN7V6pB09b4JozEIF7J8'
    chat_id = 6660799057
    bot = telepot.Bot(token)
    bot.sendMessage(chat_id, msg)
    return 0

# 홈페이지 접속
def open_browser():
    driver = webdriver.Chrome(options=options)
    return driver

#로그인
def login(driver, login_id, login_pw):
    driver.get('https://etk.srail.co.kr/cmc/01/selectLoginForm.do') # 이동을 원하는 페이지 주소 입력
    driver.implicitly_wait(15) # 페이지 다 뜰 때 까지 기다림
    driver.find_element(By.ID, 'srchDvNm01').send_keys(str(login_id)) # 회원번호
    driver.find_element(By.ID, 'hmpgPwdCphd01').send_keys(str(login_pw)) # 비밀번호
    driver.find_element(By.XPATH, '//*[@id="login-form"]/fieldset/div[1]/div[1]/div[2]/div/div[2]/input').click()
    driver.implicitly_wait(5)
    return driver

# 기차 조회
def search_train(driver, dpt_stn, arr_stn, dpt_date, dpt_time, num_trains_to_check=2, want_special=True, want_waiting=False):
    is_booked = False # 예약여부
    cnt_refresh = 0 # 시도 카운트
    driver.get('https://etk.srail.kr/hpg/hra/01/selectScheduleList.do')
    driver.implicitly_wait(5)

    #팝업닫기
    main=driver.window_handles[0]
    # driver.switch_to.window(driver.window_handles[1])
    # driver.close()
    # driver.switch_to.window(driver.window_handles[0])
    for handle in driver.window_handles:
        if handle != main:
            driver.switch_to.window(handle)
            driver.close()
    driver.switch_to.window(main)

    # 출발지 입력
    elements_dpt_stn = driver.find_element(By.ID, 'dptRsStnCdNm')
    elements_dpt_stn.clear()
    elements_dpt_stn.send_keys(dpt_stn)

    # 도착지 입력
    elements_arr_stn = driver.find_element(By.ID, 'arvRsStnCdNm')
    elements_arr_stn.clear()
    elements_arr_stn.send_keys(arr_stn)

    # 날짜 입력
    elm_dptDt = driver.find_element(By.ID, "dptDt")
    driver.execute_script("arguments[0].setAttribute('style','display: True;')", elm_dptDt)
    Select(driver.find_element(By.ID,"dptDt")).select_by_value(dpt_date)

    # 시간선택
    elm_dptTm = driver.find_element(By.ID, "dptTm")
    driver.execute_script("arguments[0].setAttribute('style','display: True;')", elm_dptTm)
    Select(driver.find_element(By.ID, "dptTm")).select_by_visible_text(dpt_time)

    # SRT 선택
    driver.find_element(By.ID,"trnGpCd300").click()

    tele("기차를 조회합니다")
    tele(f"\n출발 : {dpt_stn}, 도착 : {arr_stn}\n날짜 : {dpt_date}, 시간 : {dpt_time}시 이후")

    # 조회하기
    driver.find_element(By.XPATH,"//input[@value='조회하기']").click()
    train_list = driver.find_elements(By.CSS_SELECTOR, '#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr')

    # 리스트 확인
    for i in range(1, len(train_list)+1):
        train_list_txt = []
        for j in range(3, 8):
            text = driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child({j})").text.replace("\n"," ")
            #print(text, end="")
            train_list_txt.append(text)
        tele(str(train_list_txt))

    tele(f"위 목록에서 상위 {num_trains_to_check}개의 기차 예약을 시도합니다.")
    tele(f"특실 예약 : {want_special}, 예약 대기 : {want_waiting}\n")

    # 예약하기
    while True:
        for i in range(1, num_trains_to_check+1):
            # td:nth-child(6)이 특실, td:nth-child(7)이 일반실, td:nth-child(8)이 예약대기
            special_seat = driver.find_element(By.CSS_SELECTOR,
                                                f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(6)").text
            standard_seat = driver.find_element(By.CSS_SELECTOR,
                                                f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(7)").text
            waiting_seat = driver.find_element(By.CSS_SELECTOR,
                                               f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(8)").text

            if want_special: # 특실
                if "예약하기" in special_seat:
                    tele("특실 예약 가능. 예약을 시도합니다.")
                    driver.find_element(By.XPATH,
                                        f"/html/body/div[1]/div[4]/div/div[3]/div[1]/form/fieldset/div[6]/table/tbody/tr[{i}]/td[6]/a/span").click()
                    if driver.find_elements(By.ID, 'isFalseGotoMain'):
                        is_booked = True
                        tele("예약이 완료되었습니다. 20분 내에 결제해 주세요.")
                        break
                    else:
                        tele("잔여석 없음. 다시 시도합니다.")
                        driver.back()
                        driver.implicitly_wait(4)

            if "예약하기" in standard_seat: # 일반실
                tele("일반실 예약 가능. 예약을 시도합니다.")
                driver.find_element(By.XPATH,
                                    f"/html/body/div[1]/div[4]/div/div[3]/div[1]/form/fieldset/div[6]/table/tbody/tr[{i}]/td[7]/a/span").click()
                if driver.find_elements(By.ID, 'isFalseGotoMain'):
                    is_booked = True
                    tele("예약이 완료되었습니다. 20분 내에 결제해 주세요.")
                    break
                else:
                    tele("잔여석 없음. 다시 시도합니다.")
                    driver.back()
                    driver.implicitly_wait(4)

            if want_waiting: # 예약대기
                if "신청하기" in waiting_seat:
                    tele("예약 대기")
                    driver.find_element(By.XPATH,
                                        f"/html/body/div[1]/div[4]/div/div[3]/div[1]/form/fieldset/div[6]/table/tbody/tr[{i}]/td[8]/a/span").click()
                    is_booked = True
                    break

        if not is_booked:
            try:
            # 조회 다 될 때까지 대기. 최대 1분
            #WebDriverWait(driver, 60).until(expected_conditions.element_to_be_clickable(By.XPATH, "//input[@value='조회하기']"))
            # 3~7초 기다리기
                time.sleep(randint(3,7))
                # 다시 조회하기
                submit = driver.find_element(By.XPATH, "//input[@value='조회하기']")
            except:
                time.sleep(60)
                pass
            else:
                submit = driver.find_element(By.XPATH, "//input[@value='조회하기']")
                driver.execute_script("arguments[0].click();", submit)
                cnt_refresh += 1
                #tele(f"{cnt_refresh}회 시도...다시 시도합니다")
                print(f"{cnt_refresh}회 시도...다시 시도합니다")
                driver.implicitly_wait(10)
                time.sleep(1)
        else:
            break
    return driver

if __name__ == "__main__":
    driver = open_browser()
    driver = login(driver, '1796204911', '6cjs3qor16')
    search_train(driver, "수서", "대전", "20230926", "22", want_special=True, want_waiting=False)

1회 시도...다시 시도합니다
2회 시도...다시 시도합니다
3회 시도...다시 시도합니다
4회 시도...다시 시도합니다
5회 시도...다시 시도합니다
6회 시도...다시 시도합니다
7회 시도...다시 시도합니다
8회 시도...다시 시도합니다
9회 시도...다시 시도합니다
10회 시도...다시 시도합니다
11회 시도...다시 시도합니다
12회 시도...다시 시도합니다
13회 시도...다시 시도합니다
14회 시도...다시 시도합니다
15회 시도...다시 시도합니다
16회 시도...다시 시도합니다
17회 시도...다시 시도합니다
18회 시도...다시 시도합니다
19회 시도...다시 시도합니다
20회 시도...다시 시도합니다
21회 시도...다시 시도합니다
22회 시도...다시 시도합니다
23회 시도...다시 시도합니다
24회 시도...다시 시도합니다
25회 시도...다시 시도합니다
26회 시도...다시 시도합니다
27회 시도...다시 시도합니다
28회 시도...다시 시도합니다
29회 시도...다시 시도합니다
30회 시도...다시 시도합니다
31회 시도...다시 시도합니다
32회 시도...다시 시도합니다
33회 시도...다시 시도합니다
34회 시도...다시 시도합니다
35회 시도...다시 시도합니다
36회 시도...다시 시도합니다
37회 시도...다시 시도합니다
38회 시도...다시 시도합니다
39회 시도...다시 시도합니다
40회 시도...다시 시도합니다
41회 시도...다시 시도합니다
42회 시도...다시 시도합니다
43회 시도...다시 시도합니다
44회 시도...다시 시도합니다
45회 시도...다시 시도합니다
46회 시도...다시 시도합니다
47회 시도...다시 시도합니다
48회 시도...다시 시도합니다
49회 시도...다시 시도합니다
50회 시도...다시 시도합니다
51회 시도...다시 시도합니다
52회 시도...다시 시도합니다
53회 시도...다시 시도합니다
54회 시도...다시 시도합니다
55회 시도...다시 시도합니다
56회 시도...다시 시도합니다
5

StaleElementReferenceException: ignored